# Import Libraries

In [ ]:
from selenium import webdriver 
from bs4 import BeautifulSoup
from time import sleep
import sqlite3
from sqlite3 import Error

# SQLite Functions

In [ ]:
def create_connection(db_file):
    conn = None
    try:
        conn = sqlite3.connect(db_file)
        return conn
    except Error as e:
        print(e)

In [ ]:
def create_table(conn, create_table_sql):
    try:
        c = conn.cursor()
        c.execute(create_table_sql)
    except Error as e:
        print(e)

In [ ]:
def create_linkedin_profiles_table(conn):
    sql_create_linkedin_profiles_table = """ CREATE TABLE IF NOT EXISTS linkedin_profiles (
                                        id integer PRIMARY KEY,
                                        name text NOT NULL,
                                        headline text
                                    ); """
    if conn is not None:
        create_table(conn, sql_create_linkedin_profiles_table)

    else:
        print("Error! cannot create the database connection.")

In [ ]:
def insert_linkedin_profile(conn, profile):
    sql = ''' INSERT INTO linkedin_profiles(name,headline)
              VALUES(?,?) '''
    cur = conn.cursor()
    cur.execute(sql, profile)
    conn.commit()
    return cur.lastrowid

In [ ]:
def close_connection(conn):
    conn.close()

# Logfile

In [ ]:
def open_logfile(path):
    return open(path,'a',encoding='utf-8')

def insert_to_logfile(file,profile):
    global profile_counter
    profile_counter += 1
    file.write(f"{profile_counter},\"{profile[0]}\",\"{profile[1]}\"\n")
        
def close_logfile(file):
    file.close()

# Linkedin Functions

In [ ]:
def login_to_linkedin():
    driver.get("https://www.linkedin.com")
    driver.implicitly_wait(10)
    username = driver.find_element_by_class_name('input__input')
    username.send_keys('hamidreza.samsami@gmail.com')
    password = driver.find_element_by_id('session_password')
    password.send_keys('1qaz!QAZ') 
    log_in_button = driver.find_element_by_class_name('sign-in-form__submit-button') 
    log_in_button.click()

In [ ]:
def parse_current_page():
    html = driver.page_source
    return BeautifulSoup(html, "html.parser")

In [ ]:
def get_my_profile_info(soupified):
    my_name = soupified.find("img", {"class": "global-nav__me-photo"})['alt'].strip()
    my_headline = soupified.find("p", {"class": "identity-headline"}).get_text().strip()
    return (my_name,my_headline)

# Setup 

In [ ]:
PATH = "C:\\Program Files (x86)\\chromedriver.exe" 
driver = webdriver.Chrome(PATH) 

In [ ]:
profile_counter = 0
conn = create_connection(r"C:\Users\root\Desktop\pythonsqlite.db")
logfile_path = r'C:\Users\root\Desktop\linkedin.log'
create_linkedin_profiles_table(conn)
logfile=open_logfile(logfile_path)

In [ ]:
login_to_linkedin()

# Extract and Store My Info

In [ ]:
soupified = parse_current_page()
my_profile = get_my_profile_info(soupified)
insert_to_logfile(logfile,my_profile)
insert_linkedin_profile(conn,my_profile)

# Extract and Store My Connections info

In [ ]:
for i in range(1,101,1):
    print(f"page {i} is loading ...")
    driver.get(f"https://www.linkedin.com/search/results/people/?network=%5B%22F%22%5D&origin=MEMBER_PROFILE_CANNED_SEARCH&page={i}&sid=5v*")
    driver.implicitly_wait(10)
    soupified = parse_current_page()
    connection_li_list = soupified.find_all("li", {"class": "reusable-search__result-container"}) 
    for item in connection_li_list:
        sleep(1)
        name = item.find("span",{"aria-hidden":"true"}).text.strip()
        headline = item.find("div",{"class":"entity-result__primary-subtitle t-14 t-black t-normal"}).text.strip()
        profile = (name, headline)
        insert_to_logfile(logfile,profile)
        insert_linkedin_profile(conn,profile)

# End Program

In [ ]:
close_connection(conn)
close_logfile(logfile)